In [ ]:
%matplotlib inline

In [1]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 4.2 MB/s 


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import pandas as  pd

import random
import os

import pickle5 as pickle
import pandas as pd
import math


In [8]:
class config:
    seed = 42

In [9]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [10]:
seed_everything(config.seed)

In [11]:
batch_size = 1

In [12]:
with open('/content/drive/MyDrive/imagenet/32/train_data', 'rb') as fo:
    d = pickle.load(fo)
x = d['data']
y = d['labels']  

x_train  =  torch.FloatTensor(x)  
y_train  =  torch.LongTensor(y)

trainset0 = torch.utils.data.TensorDataset(x_train, y_train)
trainloader0 = torch.utils.data.DataLoader(trainset0, batch_size=batch_size, shuffle=False)



In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [16]:
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:
def training(model,trainloder_x) :
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()
    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloder_x, 0):
            # get the inputs; data is a list of [inputs, labels]
            #inputs, labels = data
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')   

In [18]:
training(net,trainloader0)

[1,  2000] loss: 2.304
[1,  4000] loss: 2.284
[1,  6000] loss: 2.185
[1,  8000] loss: 2.082
[1, 10000] loss: 2.076
[1, 12000] loss: 2.028
[2,  2000] loss: 1.989
[2,  4000] loss: 1.961
[2,  6000] loss: 1.933
[2,  8000] loss: 1.900
[2, 10000] loss: 1.897
[2, 12000] loss: 1.830
[3,  2000] loss: 1.855
[3,  4000] loss: 1.835
[3,  6000] loss: 1.820
[3,  8000] loss: 1.800
[3, 10000] loss: 1.819
[3, 12000] loss: 1.753
[4,  2000] loss: 1.780
[4,  4000] loss: 1.767
[4,  6000] loss: 1.754
[4,  8000] loss: 1.729
[4, 10000] loss: 1.772
[4, 12000] loss: 1.686
[5,  2000] loss: 1.737
[5,  4000] loss: 1.703
[5,  6000] loss: 1.698
[5,  8000] loss: 1.680
[5, 10000] loss: 1.707
[5, 12000] loss: 1.632
[6,  2000] loss: 1.674
[6,  4000] loss: 1.661
[6,  6000] loss: 1.646
[6,  8000] loss: 1.636
[6, 10000] loss: 1.669
[6, 12000] loss: 1.610
[7,  2000] loss: 1.639
[7,  4000] loss: 1.588
[7,  6000] loss: 1.649
[7,  8000] loss: 1.615
[7, 10000] loss: 1.614
[7, 12000] loss: 1.563
[8,  2000] loss: 1.622
[8,  4000] 

In [19]:
PATH = '/content/drive/MyDrive/Colab Notebooks/졸업논문/imagenet32_net.pth'
#torch.save(net.state_dict(), PATH)

In [20]:
with open('/content/drive/MyDrive/imagenet/32/val_data', 'rb') as fo:
    d = pickle.load(fo)
x = d['data']
y = d['labels']    

x_val  =  torch.FloatTensor(x)  
y_val  =  torch.LongTensor(y)

dataset_val = torch.utils.data.TensorDataset(x_val, y_val)
testloader = torch.utils.data.DataLoader(dataset_val, batch_size=1, shuffle=True)
        

In [21]:
def evalmodel(model, dataloader) :
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            #images, labels = data
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(100 * correct / total)

In [22]:
evalmodel(net,trainloader0)
evalmodel(net,testloader)

48.28793774319066
40.2
